In [1]:
import numpy as np
import pandas as pd

In [2]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])

In [3]:
def H(y):
    if len(y) ==0:
        return 0
    p_0=len(y[y==1])/len(y)

    if p_0==0 or p_0==1:
        return 0
    else:
        return -(p_0*np.log2(p_0)+(1-p_0)*np.log2(1-p_0))

In [5]:
print("Entropy at root node: ", H(y_train)) 

Entropy at root node:  1.0


In [7]:
def  split_dataset(X,node_indices,feature):
    left_indices=[]
    right_indices=[]

    for i in node_indices:
        if X[i][feature]==1:
            left_indices.append(i)
        else:
            right_indices.append(i)
    return left_indices,right_indices

In [12]:
root_indices=np.arange(10)
feature = 0

left_indices, right_indices = split_dataset(X_train, root_indices, feature)


print("Left indices: ", left_indices)
print("Right indices: ", right_indices)

Left indices:  [0, 1, 2, 3, 4, 7, 9]
Right indices:  [5, 6, 8]


In [10]:
def compute_information_gain(x,y,node_indices,feature):
    left_indices,right_indices=split_dataset(x,node_indices,feature)

    x_node,y_node=x[node_indices],y[node_indices]
    x_left,y_left=x[left_indices],y[left_indices]
    x_right,y_right=x[right_indices],y[right_indices]

    H_node=H(y_node)
    H_left=H(y_left)
    H_right=H(y_right)

    w_left=len(y_left)/len(y_node)
    w_right=len(y_right)/len(y_node)

    return H_node-(H_left*w_left+H_right*w_right)

In [11]:
info_gain0 = compute_information_gain(X_train, y_train, root_indices, feature=0)
print("Information Gain from splitting the root on brown cap: ", info_gain0)

info_gain1 = compute_information_gain(X_train, y_train, root_indices, feature=1)
print("Information Gain from splitting the root on tapering stalk shape: ", info_gain1)

info_gain2 = compute_information_gain(X_train, y_train, root_indices, feature=2)
print("Information Gain from splitting the root on solitary: ", info_gain2)

Information Gain from splitting the root on brown cap:  0.034851554559677034
Information Gain from splitting the root on tapering stalk shape:  0.12451124978365313
Information Gain from splitting the root on solitary:  0.2780719051126377


In [15]:
def get_best_split(x,y,node_indices):

    features=x.shape[1]
    max_gain=0
    selected_feature=-1
    for feature in range(features):
        info_gain=compute_information_gain(x,y,node_indices,feature)
        if info_gain>max_gain:
            max_gain=info_gain
            selected_feature=feature

    return selected_feature


In [16]:
best_feature = get_best_split(X_train, y_train, root_indices)
print("Best feature to split on: %d" % best_feature)

Best feature to split on: 2


In [17]:
tree = []

def construct_tree(X,y,node_indices,branch_name,max_depth,current_depth):

    if max_depth == current_depth :
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return
    
    best_split=get_best_split(X,y,node_indices)

    left_indices,right_indices=split_dataset(X,node_indices,best_split)
    tree.append((left_indices, right_indices, best_feature))

    formatting = "-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))

    construct_tree(X,y,left_indices,'left',max_depth,current_depth+1)
    construct_tree(X,y,right_indices,'right',max_depth,current_depth+1)



In [18]:
construct_tree(X_train, y_train, root_indices, "Root", max_depth=2, current_depth=0)

 Depth 0, Root: Split on feature: 2
- Depth 1, left: Split on feature: 2
  -- left leaf node with indices [0, 1, 4, 7]
  -- right leaf node with indices [5]
- Depth 1, right: Split on feature: 2
  -- left leaf node with indices [8]
  -- right leaf node with indices [2, 3, 6, 9]
